In [3]:
import numpy as np
import pandas as pd
import os
from glob import glob
from pprint import pprint
import json
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.pyplot as plt

import cellcycle.PlottingTools as plottingTools
from cellcycle.ParameterSet import ParameterSet
import cellcycle.DataStorage as dataStorage
import cellcycle.DataAnalysis as dataAnalysis
import cellcycle.MakeDataframe as makeDataframe

In [4]:
file_path = '/home/berger/Documents/Arbeit/PhD/data/UltrasensitivityCombined/results/paper/Fig_3B'

parameter_path = os.path.join(file_path, 'parameter_set.csv')

# Make data frame from time traces

In [5]:
data_frame = makeDataframe.make_dataframe(file_path)
time_traces_data_frame = pd.read_hdf(data_frame['path_dataset'].iloc[0], key='dataset_time_traces')

v_init_data_frame = pd.read_hdf(data_frame['path_dataset'].iloc[0], key='dataset_init_events')
v_init = v_init_data_frame.iloc[0]['v_init']
t_init_list = v_init_data_frame['t_init'].to_numpy()
print(v_init)

v_d_data_frame = pd.read_hdf(data_frame['path_dataset'].iloc[0], key='dataset_div_events')
print(v_d_data_frame['v_b'])


['/home/berger/Documents/Arbeit/PhD/data/UltrasensitivityCombined/results/paper/Fig_3B/Fig_3B_presentation.pdf',
 '/home/berger/Documents/Arbeit/PhD/data/UltrasensitivityCombined/results/paper/Fig_3B/LD_100_0.5',
 '/home/berger/Documents/Arbeit/PhD/data/UltrasensitivityCombined/results/paper/Fig_3B/Fig_3B_old.pdf',
 '/home/berger/Documents/Arbeit/PhD/data/UltrasensitivityCombined/results/paper/Fig_3B/Fig_3B.pdf',
 '/home/berger/Documents/Arbeit/PhD/data/UltrasensitivityCombined/results/paper/Fig_3B/Fig_3B_new.pdf']
0.28094848599459255
0    0.198786
1    0.198762
2    0.198808
3    0.198784
4    0.198760
5    0.198805
6    0.198781
7    0.198757
Name: v_b, dtype: float64


In [6]:
time = np.array(time_traces_data_frame["time"])
volume = np.array(time_traces_data_frame["volume"])
n_ori = np.array(time_traces_data_frame["n_ori"])
active_fraction = np.array(time_traces_data_frame["active_fraction"])
print(time.size)
t_0 = time[volume==v_d_data_frame['v_b'][0]]
indx_0 = np.where(time==t_0)[0][0]
t_f = time[volume==v_d_data_frame['v_b'][3]]
indx_f = np.where(time==t_f)[0][0]+20

print(indx_0, indx_f)

n_ori_cut = n_ori[indx_0:indx_f]
time_cut = time[indx_0:indx_f]
volume_cut = volume[indx_0:indx_f]
active_fraction_cut = active_fraction[indx_0:indx_f]

t_init_list_cut_1 = t_init_list[t_init_list>t_0]
t_init_list_cut = t_init_list_cut_1[t_init_list_cut_1<t_f]
t_b = t_init_list_cut + data_frame.iloc[0]['t_CD']
t_b_cut = t_b[t_b<t_f]

20000
2812 8833


# Color definitions

In [7]:
pinkish_red = (247 / 255, 109 / 255, 109 / 255)
green = (0 / 255, 133 / 255, 86 / 255)
dark_blue = (36 / 255, 49 / 255, 94 / 255)
light_blue = (168 / 255, 209 / 255, 231 / 255)
blue = (55 / 255, 71 / 255, 133 / 255)
yellow = (247 / 255, 233 / 255, 160 / 255)

# Plot three figures

In [8]:
label_list = [r'$V(t)$', r'$n_{ori}(t)$', r'$f(t)$']
x_axes_list = [time_cut, time_cut, time_cut]
y_axes_list = [volume_cut, n_ori_cut, active_fraction_cut]
color_list = [green, dark_blue, pinkish_red]
fig, ax = plt.subplots(3, figsize=(3.2,4))
plt.xlabel(r'time [$\tau_{\rm d}$]')
y_min_list = [0,0,0,0]
y_max_list = [1, 1.2, 1.2, 1.2]

doubling_time = 1/data_frame.iloc[0]['doubling_rate']
for item in range(0, len(label_list)):
    ax[item].set_ylabel(label_list[item])
    ax[item].plot(x_axes_list[item], y_axes_list[item], color=color_list[item])
    ax[item].set_ylim(ymin=0)
    ax[item].tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
    ax[item].spines["top"].set_visible(False)
    ax[item].spines["right"].set_visible(False)
    ax[item].margins(0)
    for t_div in t_b_cut:
        ax[item].axvline(x=t_div, 
                          ymin=y_min_list[item],
                          ymax=y_max_list[item],
                             c="black",
                          zorder=0, 
                          linewidth=0.8,
                          clip_on=False)
    for t_init in t_init_list_cut:
        ax[item].axvline(x=t_init, 
                          ymin=y_min_list[item],
                          ymax=y_max_list[item],
                             c="black",
                          zorder=0, 
                          linewidth=0.8,
                          linestyle='--',
                          clip_on=False)
ax[0].set_yticks([0,v_init,0.4])
ax[0].set_yticklabels(['0',r'$v^\ast$', '0.4'])
ax[0].get_yticklabels()[1].set_color(green)
ax[0].axhline(y=v_init, color=green, linestyle='--')

ax[1].set_yticks([0,1,2])
ax[1].set_yticklabels([r'0',r'1', r'2'])

ax[2].axhline(y=data_frame.iloc[0]['frac_init'], color=pinkish_red, linestyle='--')
# ax[2].set_yticks([0, data_frame.iloc[0]['frac_init']])
# ax[2].set_yticklabels(['0', r'$f^\ast$'])
# ax[2].get_yticklabels()[1].set_color(pinkish_red)
ax[2].set_yticks([0, 0.5, data_frame.iloc[0]['frac_init'], 1])
ax[2].set_yticklabels(['0', '0.5', r'$f^\ast$', '1'])
ax[2].get_yticklabels()[2].set_color(pinkish_red)
ax[2].tick_params(bottom=True, labelbottom=True)
ax[2].tick_params(axis='x', colors='black')
ax[2].set_xticks([time_cut[0], 
                  time_cut[0]+ doubling_time, 
                  time_cut[0]+ 2*doubling_time,
                  time_cut[0]+ 3*doubling_time
                 ])
ax[2].set_xticklabels(['0', '1', '2', '3'])
plt.savefig(file_path + '/Fig_3B.pdf', format='pdf', bbox_inches='tight')